# 最大熵模型中文分词

In [89]:
import sys
import re
import logging

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)
logger.info('Hello world!')

INFO:__main__:Hello world!


## 给句子中的每个字符打标签

In [37]:
def tag4_word(word):
    """
    tag4给词语中的每个字符打标签
    """
    tag_word = ''
    if len(word) == 0:
        return ''
    
    if len(word) == 1:
        tag_word = word + '/S'
    elif len(word) == 2:
        tag_word = word[0] + '/B' + word[1] + '/E'
    else:
        tag_word = word[0] + '/B'
        for char in word[1:-1]:
            tag_word = tag_word + char + '/M'
        tag_word = tag_word + word[-1] + '/E'
    return tag_word

def tag4_sentence(sentence):
    """
    tag4给句子中的每个字符打标签
    sentence: 空格分隔个句子。e.g. "我 爱 伟大 的 中国 。"
    """
    words = re.split('\s+', sentence)
    
    tag_word_list = []
    for word in words:
        tag_word = tag4_word(word)
        tag_word_list.append(tag_word)
    return ''.join(tag_word_list)

def detag4_sentence(sentence):
    """
    对tag4打标签的句子进行反解码，输出按空格分隔的句子
    """
    if len(sentence) == 0:
        return ''
    
    sen = sentence.replace('/S', ' ').replace('/E', ' ').replace('/B', '').replace('/M', '')
    return sen

In [114]:
def tag4_data(input_file, output_file):
    """
    对读入文件中的句子打tag4标签，并写出文件
    """
    with open(input_file, 'r') as f:
        train_data = f.readlines()
        train_data_row_num = len(train_data)
        with open(output_file, 'w+') as f:
            f.truncate()
            for row, sentence in enumerate(train_data):
                tag4_sent = tag4_sentence(sentence)
                f.write(tag4_sent + '\n')
                logger.info('写入tag4标签[%d/%d]. [sentence]=%s, [tag4-sentence]=%s' % (row+1, train_data_row_num, sentence, tag4_sent))
        logger.info('成功将tag4标签写入文件 ')
                
tag4_data('../data/pku_training10.txt', '../data/pku_tag4_training10.txt')

INFO:__main__:写入tag4标签[1/9]. [sentence]=迈向  充满  希望  的  新  世纪  ——  一九九八年  新年  讲话  （  附  图片  １  张  ）  
, [tag4-sentence]=迈/B向/E充/B满/E希/B望/E的/S新/S世/B纪/E—/B—/E一/B九/M九/M八/M年/E新/B年/E讲/B话/E（/S附/S图/B片/E１/S张/S）/S
INFO:__main__:写入tag4标签[2/9]. [sentence]=中共中央  总书记  、  国家  主席  江  泽民  
, [tag4-sentence]=中/B共/M中/M央/E总/B书/M记/E、/S国/B家/E主/B席/E江/S泽/B民/E
INFO:__main__:写入tag4标签[3/9]. [sentence]=（  一九九七年  十二月  三十一日  ）  
, [tag4-sentence]=（/S一/B九/M九/M七/M年/E十/B二/M月/E三/B十/M一/M日/E）/S
INFO:__main__:写入tag4标签[4/9]. [sentence]=１２月  ３１日  ，  中共中央  总书记  、  国家  主席  江  泽民  发表  １９９８年  新年  讲话  《  迈向  充满  希望  的  新  世纪  》  。  （  新华社  记者  兰  红光  摄  ）  
, [tag4-sentence]=１/B２/M月/E３/B１/M日/E，/S中/B共/M中/M央/E总/B书/M记/E、/S国/B家/E主/B席/E江/S泽/B民/E发/B表/E１/B９/M９/M８/M年/E新/B年/E讲/B话/E《/S迈/B向/E充/B满/E希/B望/E的/S新/S世/B纪/E》/S。/S（/S新/B华/M社/E记/B者/E兰/S红/B光/E摄/S）/S
INFO:__main__:写入tag4标签[5/9]. [sentence]=同胞  们  、  朋友  们  、  女士  们  、  先生  们  ：  
, [tag4-sentence]=同/B胞/E们/S、/S朋/B友/E们/S、/S女/B士/E们/S、/S先/B生/E们/S：/S
INFO:__main__:写入tag4标签[6/9]. [sentence]=

## 定义特征函数

In [121]:
def get_char(sentence, i, steps=3):
    """
    获取sentence在位置i处的字符
    """
    words_len = len(sentence) / steps;
    if (i < 0 or i > words_len - 1):
        return '_'
    else:
        return sentence[i*steps]
    
def get_feature1(sentence, i, steps=3):
    """
    特征1：Cn(n=-2,-1,0,1,2)
    """
    words = []
    words.append('C-2=' + get_char(sentence, i - 2, steps))
    words.append('C-1=' + get_char(sentence, i - 1, steps))
    words.append('C0=' + get_char(sentence, i, steps))
    words.append('C1=' + get_char(sentence, i + 1, steps))
    words.append('C2=' + get_char(sentence, i + 2, steps))
    return ''.join(words)

def feature_sentence(sentence, steps=3):
    features = []
    for i in range(len(sentence)//steps):
        # 特征1
        feat = get_feature1(sentence, i, steps)
        # 标签
        label = sentence[i*steps+2]
        if len(feat) > 0:
            features.append((feat, label))
    return features

feature_sentence('是/S中/B国/E发/B展/E历/B史/E上/S非/B常/E重/B要/E的/S', 3)

[('C-2=_C-1=_C0=是C1=中C2=国', 'S'),
 ('C-2=_C-1=是C0=中C1=国C2=发', 'B'),
 ('C-2=是C-1=中C0=国C1=发C2=展', 'E'),
 ('C-2=中C-1=国C0=发C1=展C2=历', 'B'),
 ('C-2=国C-1=发C0=展C1=历C2=史', 'E'),
 ('C-2=发C-1=展C0=历C1=史C2=上', 'B'),
 ('C-2=展C-1=历C0=史C1=上C2=非', 'E'),
 ('C-2=历C-1=史C0=上C1=非C2=常', 'S'),
 ('C-2=史C-1=上C0=非C1=常C2=重', 'B'),
 ('C-2=上C-1=非C0=常C1=重C2=要', 'E'),
 ('C-2=非C-1=常C0=重C1=要C2=的', 'B'),
 ('C-2=常C-1=重C0=要C1=的C2=_', 'E'),
 ('C-2=重C-1=要C0=的C1=_C2=_', 'S')]

In [123]:
def feature_data(input_file, output_file):
    steps = 3
    with open(input_file, 'r') as fin:
        with open(output_file, 'w+') as fout:
            fout.truncate()
            for row in fin.readlines():
                row = row.strip()
                # 提取特征
                features = feature_sentence(row, steps)
                if len(features) == 0: continue
                feature_str = '\n'.join([fea[0] + ' ' + fea[1] for fea in features])
                # 写入文本
                fout.write(feature_str)
                logger.info('写入特征. [feature]=%s'% feature_str)
        logger.info('成功将特征写入文件')

feature_data('../data/pku_tag4_training10.txt', '../data/pku_feature_training10.txt')

INFO:__main__:写入特征. [feature]=C-2=_C-1=_C0=迈C1=向C2=充 B
C-2=_C-1=迈C0=向C1=充C2=满 E
C-2=迈C-1=向C0=充C1=满C2=希 B
C-2=向C-1=充C0=满C1=希C2=望 E
C-2=充C-1=满C0=希C1=望C2=的 B
C-2=满C-1=希C0=望C1=的C2=新 E
C-2=希C-1=望C0=的C1=新C2=世 S
C-2=望C-1=的C0=新C1=世C2=纪 S
C-2=的C-1=新C0=世C1=纪C2=— B
C-2=新C-1=世C0=纪C1=—C2=— E
C-2=世C-1=纪C0=—C1=—C2=一 B
C-2=纪C-1=—C0=—C1=一C2=九 E
C-2=—C-1=—C0=一C1=九C2=九 B
C-2=—C-1=一C0=九C1=九C2=八 M
C-2=一C-1=九C0=九C1=八C2=年 M
C-2=九C-1=九C0=八C1=年C2=新 M
C-2=九C-1=八C0=年C1=新C2=年 E
C-2=八C-1=年C0=新C1=年C2=讲 B
C-2=年C-1=新C0=年C1=讲C2=话 E
C-2=新C-1=年C0=讲C1=话C2=（ B
C-2=年C-1=讲C0=话C1=（C2=附 E
C-2=讲C-1=话C0=（C1=附C2=图 S
C-2=话C-1=（C0=附C1=图C2=片 S
C-2=（C-1=附C0=图C1=片C2=１ B
C-2=附C-1=图C0=片C1=１C2=张 E
C-2=图C-1=片C0=１C1=张C2=） S
C-2=片C-1=１C0=张C1=）C2=_ S
C-2=１C-1=张C0=）C1=_C2=_ S
INFO:__main__:写入特征. [feature]=C-2=_C-1=_C0=中C1=共C2=中 B
C-2=_C-1=中C0=共C1=中C2=央 M
C-2=中C-1=共C0=中C1=央C2=总 M
C-2=共C-1=中C0=央C1=总C2=书 E
C-2=中C-1=央C0=总C1=书C2=记 B
C-2=央C-1=总C0=书C1=记C2=、 M
C-2=总C-1=书C0=记C1=、C2=国 E
C-2=书C-1=记C0=、C1=国C2=家 S
C-2=记C-1=、C0=国C1=家C2=主 B
C-2=、C-1=国C0=家C

## 训练最大熵模型

In [124]:
from nltk.classify import MaxentClassifier

In [136]:
def parse_tag4_to_joint_feature(string):
    return ({'C-2': string[4], 'C-1': string[9], 'C0': string[13], 'C1': string[17], 'C2': string[21]}, string[-1])

# 载入训练数据
with open('../data/pku_feature_training10.txt', 'r') as f:
    train_features = [parse_tag4_to_joint_feature(row.strip()) for row in f.readlines()]
train_features

[({'C-1': '_', 'C-2': '_', 'C0': '迈', 'C1': '向', 'C2': '充'}, 'B'),
 ({'C-1': '迈', 'C-2': '_', 'C0': '向', 'C1': '充', 'C2': '满'}, 'E'),
 ({'C-1': '向', 'C-2': '迈', 'C0': '充', 'C1': '满', 'C2': '希'}, 'B'),
 ({'C-1': '充', 'C-2': '向', 'C0': '满', 'C1': '希', 'C2': '望'}, 'E'),
 ({'C-1': '满', 'C-2': '充', 'C0': '希', 'C1': '望', 'C2': '的'}, 'B'),
 ({'C-1': '希', 'C-2': '满', 'C0': '望', 'C1': '的', 'C2': '新'}, 'E'),
 ({'C-1': '望', 'C-2': '希', 'C0': '的', 'C1': '新', 'C2': '世'}, 'S'),
 ({'C-1': '的', 'C-2': '望', 'C0': '新', 'C1': '世', 'C2': '纪'}, 'S'),
 ({'C-1': '新', 'C-2': '的', 'C0': '世', 'C1': '纪', 'C2': '—'}, 'B'),
 ({'C-1': '世', 'C-2': '新', 'C0': '纪', 'C1': '—', 'C2': '—'}, 'E'),
 ({'C-1': '纪', 'C-2': '世', 'C0': '—', 'C1': '—', 'C2': '一'}, 'B'),
 ({'C-1': '—', 'C-2': '纪', 'C0': '—', 'C1': '一', 'C2': '九'}, 'E'),
 ({'C-1': '—', 'C-2': '—', 'C0': '一', 'C1': '九', 'C2': '九'}, 'B'),
 ({'C-1': '一', 'C-2': '—', 'C0': '九', 'C1': '九', 'C2': '八'}, 'M'),
 ({'C-1': '九', 'C-2': '一', 'C0': '九', 'C1': '八', 'C2': '年'}, '

In [137]:
# 训练最大熵模型
maxent_cls = MaxentClassifier.train(train_features)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.38629        0.219
             2          -0.68891        0.921
             3          -0.50103        0.959
             4          -0.40000        0.973
             5          -0.33491        0.985
             6          -0.28890        0.993
             7          -0.25441        0.993
             8          -0.22750        0.993
             9          -0.20587        0.994
            10          -0.18807        0.994
            11          -0.17315        0.994
            12          -0.16046        0.995
            13          -0.14953        0.996
            14          -0.14001        0.998
            15          -0.13164        0.998
            16          -0.12423        0.998
            17          -0.11762        0.999
            18          -0.11167        0.999
            19          -0.10631        0.999
 